# Moja mama optimalizuje lepšie ako tvoja : LOG REGRESIA 2023


## 1 Ťahanie Dát

In [ ]:
import numpy as np
import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import OptimizeResult

In [ ]:
!git clone https://github.com/AdrianPauer/MVOdata2023
table = pd.read_csv("MVOdata2023/credit_risk_train.csv",sep = ';')
table

Cloning into 'MVOdata2023'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), 2.72 KiB | 1.36 MiB/s, done.


,Creditability,Duration of Credit (month),Value Savings/Stocks,Length of current employment
0,0,24,1,2
1,0,36,1,5
2,0,6,4,2
3,0,12,1,3
4,0,24,3,5
...,...,...,...,...
694,0,20,1,3
695,0,9,1,3
696,0,18,1,5
697,0,24,1,2


## 2.1 Príprava na testovanie metód

### 2.1.1 Účelové funkcie 
Task A,B



In [ ]:
# f1 na ladanie metod
def f1(x, a=1):
    M = np.diag((1, a))
    h = np.array((a, a))
    return 0.5 * x@M@x - x@h

def df1 (x,a = 1):
    M = np.diag((1, a))
    h = np.array((a, a))
    return M@x - h

def J(x):
  sum = 0
  for index,row in table.iterrows():
    v = row[0]
    u = np.array(row[1:])
    x_u = x@u
    sum += x_u * (1-v) +  np.log(1 + np.exp(-x_u))
  return sum

def dfJ(x):
  sum = np.zeros_like(x)
  for index,row in table.iterrows():
    v = row[0]
    u = np.array(row[1:])
    x_u = x@u
    sum += u * (1-v) - (np.exp(-x_u)/(1 + np.exp(-x_u))) * u
  return sum 


In [ ]:

print(J(np.zeros(3)))
print(dfJ(np.zeros(3)))
print(np.exp(-0))

484.5098792114029
[-1890.5  -388.   -511. ]
1.0


##2.2 TaskC

###2.2.1 bgfs

In [ ]:
def backtracking_line_search(fun, jac, x0, args=(), s=None, max_stepsize=1,
                             rho=0.1, c=0.05, options={}):
    """
    Implements the backtracking line search algorithm to find the step size alpha
    for gradient descent.

    Parameters:
        f : callable
            Objective function.
        g : callable
            Gradient function.
        x0 : array-like
            Current point.
        s : array-like, optional
            Search direction. If not provided, gradient direction is used.
        max_stepsize : numeric, optional.
            Initial step size. Default is 1.
        rho : numeric, optional
            Shrinkage factor. Nonnegative number, less than one. Default is 0.1.
        c : numeric, optional
            Armijo condition parameter. Default is 0.05.
        options : dict, optional
            A diactionary with solver options.
                maxiter : int
                    Maximum number of iterations to perform.

    Returns
    -------
        stepsize : float
            Step size that satisfies the Armijo condition.
    """

    assert 0 < rho < 1, "Rho parameter should be from range (0, 1)!"

    maxiter = options.get("maxiter", 100)

    x = np.array(x0)
    if s is None:
        s = -jac(x, *args)
        df_s = -s@s
    else:
        df_s = jac(x, *args) @ s
    
    stepsize = max_stepsize

    fx = fun(x, *args)
  
    
    for it in range(maxiter):
        if fun(x + stepsize*s, *args) > fx + c*stepsize*df_s:
            stepsize *= rho
        else:
            return stepsize
    
    return stepsize

In [ ]:
def bfgs(fun, jac, x0, args=(), callback=None, options={}, **kwargs):
    """
    Minimization method

    Parameters
    ----------
    jac : callable f(x, *args)
        Gradient of objective function.
        
    x0 : array-like
        Initial guess. 
    
    args : tuple, optional
        Extra arguments passed to the `jac`.
    
    callback : callable f(x), optional
        Function called after each iteration.
    
    options : dict, optional
        A diactionary with solver options.
            maxiter : int
                Maximum number of iterations to perform. Default is 1000.
            tol : float
                Tolerance for termination. Default is 1e-8.
            H0 : array-like
                Initial approximation of inverse hessian. Default is
                identity matrix.
            max_stepsize: float
                Maximum step size for line search. Default is 10.

    **kwargs : dict, optional
        Other parameters passed to `bfgs`.


    Returns
    -------
    res : OptimizeResult
        The optimization result represented as a OptimizeResult object.
        Important attributes are: x the solution,
        success a Boolean flag indicating if the optimizer exited successfully,
        message which describes the cause of the termination.
        See OptimizeResult for a description of other attributes.
    """

    x = np.array(x0)
    n = len(x)

    maxiter = options.get("maxiter", 1000)
    tol = options.get("tol", 1e-8)
    H = options.get("H0", np.eye(n))
    max_stepsize = options.get("max_stepsize", 10)

    g = jac(x, *args)
    for it in range(maxiter):
        s = -H @ g

        zuzenie = lambda a: fun(x + a*s,*args)
        dzuzenie = lambda a : jac(x + a*s, *args) * s

        stepsize = backtracking_line_search(zuzenie,dzuzenie,x,args,max_stepsize = 1, s = s)
      
        x_old = x.copy()
        g_old = g.copy()

      
        x = x + stepsize * s
        #print(stepsize, x)
        g = jac(x, *args)

        y = g - g_old
        p = x - x_old

        pTy = p@y
        Hy = H@y
      
        deltaH = (1 + ( (y@H@y) / pTy ) ) * (np.outer(p,p) / pTy) - ( ( np.outer(Hy, p) + (np.outer(p,y))@H) / pTy )
        
        H = H + deltaH

        if callback is not None:
            callback(x)
        
        if np.linalg.norm(jac(x, *args)) <= tol:
            break
    
    
    success = np.linalg.norm(jac(x, *args)) <= tol

    if success:
        msg = "Optimization successful"
    else:
        msg = "Optimization failed"
    
    return OptimizeResult(x = x, nit = it + 1, njev = it + 1, success = success, message = msg)


In [ ]:
bfgs_iterations = []
fill_bgfs = lambda x: bfgs_iterations.append(x)
bfgs(fun = J, jac=dfJ, x0 = np.zeros(3), callback=fill_bgfs, options = {'tol' : 1e-3})

[0. 0. 0.] 484.5098792114029
[1.8905e-05 3.8800e-06 5.1100e-06] 485.3362021647321
[-3.48654601e-05  2.46818878e-04  2.80877403e-04] 484.1384710624063
[-0.00166498 -0.03463503  0.0366589 ] 487.39071003413574
[-0.0040801  -0.01083558  0.05264288] 465.59426886516604
[-0.03533421  0.23378575  0.2912046 ] 390.25188763915645
[-0.04203863  0.29020902  0.33144828] 388.387505293396
[-0.04506805  0.32130541  0.34644183] 388.1866051367826
[-0.04538257  0.32680964  0.34687675] 388.183417605291
[-0.04537399  0.32755619  0.34643404] 388.18335518564436
[-0.04536188  0.32767098  0.34626089] 388.18335152052856
[-0.0453602   0.32766663  0.34624663] 388.1833514830638
[-0.0453602   0.32766662  0.34624663] 388.18335148305925


 message: Optimization successful
 success: True
       x: [-4.536e-02  3.277e-01  3.462e-01]
     nit: 13
    njev: 13

###2.2.2 dfp

In [ ]:
def dfp (fun, jac, x0, args=(), callback=None, options={}, **kwargs):
    """
    Minimization method

    Parameters
    ----------
    jac : callable f(x, *args)
        Gradient of objective function.
        
    x0 : array-like
        Initial guess. 
    
    args : tuple, optional
        Extra arguments passed to the `jac`.
    
    callback : callable f(x), optional
        Function called after each iteration.
    
    options : dict, optional
        A diactionary with solver options.
            maxiter : int
                Maximum number of iterations to perform. Default is 1000.
            tol : float
                Tolerance for termination. Default is 1e-8.
            H0 : array-like
                Initial approximation of inverse hessian. Default is
                identity matrix.
            max_stepsize: float
                Maximum step size for line search. Default is 10.

    **kwargs : dict, optional
        Other parameters passed to `bfgs`.


    Returns
    -------
    res : OptimizeResult
        The optimization result represented as a OptimizeResult object.
        Important attributes are: x the solution,
        success a Boolean flag indicating if the optimizer exited successfully,
        message which describes the cause of the termination.
        See OptimizeResult for a description of other attributes.
    """

    x = np.array(x0)
    n = len(x)

    maxiter = options.get("maxiter", 1000)
    tol = options.get("tol", 1e-8)
    H = options.get("H0", np.eye(n))
    max_stepsize = options.get("max_stepsize", 10)

    g = jac(x, *args)
    for it in range(maxiter):
        s = -H @ g

        zuzenie = lambda a: fun(x + a*s,*args)
        dzuzenie = lambda a : jac(x + a*s, *args) * s

        stepsize = backtracking_line_search(zuzenie,dzuzenie,x,args,max_stepsize = 1, s = s)
        
        x_old = x.copy()
        g_old = g.copy()

        x = x + stepsize * s
        g = jac(x, *args)

        y = g - g_old
        p = x - x_old

        Hy = H@y

    
        deltaH = ( np.outer(p,p) / (p@y) ) - ( np.outer(Hy,y@H) / (y@Hy) )  

        H = H + deltaH

        if callback is not None:
            callback(x)
        
        if np.linalg.norm(jac(x,*args)) < tol:
            break
    
    
    success = np.linalg.norm(jac(x, *args)) < tol

    if success:
        msg = "Optimization successful"
    else:
        msg = "Optimization failed"
    
    return OptimizeResult(x = x, nit = it + 1, njev = it + 1, success = success, message = msg)


In [ ]:
dfp_iterations = []
fill_dfp = lambda x: dfp_iterations.append(x)
dfp(fun = J, jac=dfJ, x0 = np.zeros(3), callback=fill_dfp, options = {'tol' : 1e-3})

[0. 0. 0.] 484.5098792114029
[1.8905e-05 3.8800e-06 5.1100e-06] 485.30711393588444
[-3.31223675e-05  2.38946317e-04  2.71941039e-04] 484.1493059600406
[-0.00166272 -0.03462458  0.0366336 ] 481.80950774640746
[-0.03590396  0.24056581  0.29443872] 389.97136538853226
[-0.04214883  0.29194748  0.33175114] 388.37062120281496
[-0.0451062   0.32184355  0.34653775] 388.1860279593485
[-0.04538421  0.32685258  0.3468689 ] 388.1834137826862
[-0.04537424  0.32755837  0.3464353 ] 388.1833552248884
[-0.04536215  0.32767241  0.34626295] 388.18335153280606
[-0.04536215  0.32767241  0.34626295] 388.1833515330311


 message: Optimization successful
 success: True
       x: [-4.536e-02  3.277e-01  3.462e-01]
     nit: 11
    njev: 11

##2.3 TaskD